In [ ]:
# kernel - tvenv

In [ ]:
import os
import glob
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
import itertools
from statsmodels.tsa.stattools import adfuller
import sklearn
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [ ]:
csv_files = glob.glob("dataset/*.csv")
print(csv_files[:2])

In [ ]:
energy_df = pd.concat(pd.read_csv(file) for file in csv_files)
energy_df

In [ ]:
energy_df.info()

In [ ]:
energy_df.describe()

In [ ]:
for col in energy_df.columns:
    print(col + ": " + str(energy_df[col].nunique()))

In [ ]:
# dropping 2 columns with the same value throughout the dataframe
energy_df.drop(["REGION","PERIODTYPE"],axis=1,inplace=True)
energy_df.head()

In [ ]:
energy_df['SETTLEMENTDATE'] = pd.to_datetime(energy_df['SETTLEMENTDATE'])
energy_df.head(2)

In [ ]:
# verifying if the column has changed to datetime field
energy_df.info()

In [ ]:
energy_df.sort_values(by=["SETTLEMENTDATE"],inplace=True)
energy_df[100:105]

#### Visualizing average daily demand for the entire period

In [ ]:
# calculating average demand per day for visualization
vis_df = energy_df.copy()
vis_df['SD_DATE'] = vis_df['SETTLEMENTDATE'].dt.date
vis_df = vis_df.groupby('SD_DATE')['TOTALDEMAND'].sum().reset_index()
vis_df.head(5)

In [ ]:
vis_df = vis_df.set_index("SD_DATE")
vis_df.head(5)

In [ ]:
vis_df.plot(figsize=(15, 6))
plt.show()

In [ ]:
decomposition = sm.tsa.seasonal_decompose(vis_df,period=365)
fig = decomposition.plot()
plt.show()

#### Extracting more features for modelling

In [ ]:
energy_df['Weekday'] = energy_df['SETTLEMENTDATE'].dt.weekday
energy_df['Hour'] = energy_df['SETTLEMENTDATE'].dt.hour
energy_df['Minute'] = energy_df['SETTLEMENTDATE'].dt.minute
energy_df['Quarter'] = energy_df['SETTLEMENTDATE'].dt.quarter
energy_df['Time'] = (energy_df['Hour'] * 60 + energy_df['Minute']) / 60
energy_df['Month'] = energy_df['SETTLEMENTDATE'].dt.month

In [ ]:
# importing holidays in NSW

import holidays

energy_df['Year'] = energy_df['SETTLEMENTDATE'].dt.year
years = list(energy_df['Year'].unique())
nsw_holidays = holidays.Australia(state="NSW",years=years)
next(iter(nsw_holidays))

In [ ]:
# marking holidays in NSW
energy_df['SETTLEMENTDATE_Date'] = energy_df['SETTLEMENTDATE'].dt.date
def label_holidays(date):
    if date in nsw_holidays.keys():
        return 1
    else:
        return 0
    
energy_df['Holiday'] = energy_df['SETTLEMENTDATE_Date'].apply(label_holidays)

In [ ]:
energy_df.loc[energy_df['Holiday']==1]

In [ ]:
# dropping year and date columns as they were extracted to support other columns and not really needed
energy_df.drop(['SETTLEMENTDATE_Date','Year'],axis=1,inplace=True)
energy_df.head(5)

In [ ]:
# Adding demand lag columns
energy_df['Demand_lag1'] = energy_df['TOTALDEMAND'].shift(1)
energy_df['Demand_lag2'] = energy_df['TOTALDEMAND'].shift(2)
energy_df = energy_df.dropna()
energy_df.head(5)

In [ ]:
# checking if all columns are in the form we need
energy_df.info()

In [ ]:
# Resampling the time series data to half an hour frequency
energy_df = energy_df.set_index('SETTLEMENTDATE').resample("30min").mean().reset_index()
energy_df.head(5)

In [ ]:
energy_df = energy_df.set_index('SETTLEMENTDATE')
energy_df.head(5)

In [ ]:
decompose_plot = sm.tsa.seasonal_decompose(energy_df['TOTALDEMAND'],period=48*365)
fig = decompose_plot.plot()
fig.set_size_inches((16, 9))
fig.tight_layout()
plt.show()

#### AD Fuller test

In [ ]:
# AD Fuller test OR unit root test for stationarity
# H0 - A unit root exists for the series and data is not stationary
# H1 - The series has no unit root and data is stationary
# If p-value > 0.05, H0 holds and data is non-stationary
# If p-value < 0.05, reject HO and data is stationary

stationarity_test = adfuller(energy_df['TOTALDEMAND'])
print('ADF Statistic: %f' % stationarity_test[0])
print('p-value: %f' % stationarity_test[1])
print('Critical Values:')
for key, value in stationarity_test[4].items():
    print('\t%s: %.3f' % (key, value))


#### Univariate time series foecasting using Seasonal ARIMA
<p> This uses only previous values from the same variable to predict the next or future values, no other features or predictors are considered

In [107]:
train = energy_df.loc[:'2022-1-31 00:00:00']
test = energy_df.loc['2022-01-31 00:0:00':]

In [ ]:
model = sm.tsa.statespace.SARIMAX(train['TOTALDEMAND'], order=(1, 1, 1))
model_fit = model.fit(disp=False)

In [ ]:
test['predictions'] = model_fit.predict(start='2022-01-31 00:0:00', end=test.index[-1])

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(test['TOTALDEMAND'], test['predictions'])
mse = mean_squared_error(test['TOTALDEMAND'], test['predictions'])
rmse = np.sqrt(mse)

print(f"Mean Absolute Error: {mae:.4f}")
print(f"Mean Squared Error: {mse:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")

In [ ]:
plt.figure(figsize=(15,5))
plt.xlabel('Year')
plt.ylabel('Demand')

test['TOTALDEMAND'].plot(color='blue')
test['predictions'].plot(color='orange')
plt.show()

##### Observation

<p> After few initial forecasts, all other fpredictions give the same value.<br>
    This is because simple AR and MA models are intended to perform one-step-ahead predictions

In [ ]:
test[100:120]['predictions']

##### Attempting the same method after scaling the values

In [ ]:
scaler = MinMaxScaler()
train['DEMAND_scaled'] = scaler.fit_transform(train[['TOTALDEMAND']])
test['DEMAND_scaled'] = scaler.transform(test[['TOTALDEMAND']])

model = sm.tsa.statespace.SARIMAX(train['DEMAND_scaled'], order=(1, 1, 1))
model_fit = model.fit(disp=False)

test['predictions'] = model_fit.predict(start='2022-01-31 00:0:00', end=test.index[-1])

mae = mean_absolute_error(test['DEMAND_scaled'], test['predictions'])
mse = mean_squared_error(test['DEMAND_scaled'], test['predictions'])
rmse = np.sqrt(mse)

print(f"Mean Absolute Error: {mae:.4f}")
print(f"Mean Squared Error: {mse:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")

#### Recursive multi-step forecasting
1. Train a single model to predict demand for next time step
2. Model is trained iteratively with predicted demand to get predictions for multiple timesteps

In [120]:
train = energy_df.loc[:'2023-06-27 14:00:00']
test = energy_df.loc['2023-06-27 14:30:00':]
test_idx = '2023-06-27 14:30:00'
ts=pd.Series(to_predict.index,index=to_predict.index).sort_index().shift(-1)
train['DEMAND_FORECAST'] = energy_df['TOTALDEMAND']
test['DEMAND_FORECAST'] = 0.0
for i in range(test.shape[0]):
    print("Training details:")
    print("Train set size", train.shape[0])
    print("Predicting demand for timestamp:", test_idx)
    model = sm.tsa.statespace.SARIMAX(train['DEMAND_FORECAST'], order=(1, 1, 1))
    model_fit = model.fit(disp=False)
    predicion_row = test.loc[test_idx]
    prediction = model_fit.predict(start=test_idx, end=test_idx)
    test.loc[test.index == test_idx, 'DEMAND_FORECAST'] = prediction[test_idx]
    train = pd.concat([train, test.loc[test.index == test_idx]])
    test_idx = ts[test_idx]
    

/var/folders/7w/p9vq5jld0ys_125td69sqz_h0000gn/T/ipykernel_4484/342759256.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['DEMAND_FORECAST'] = energy_df['TOTALDEMAND']
/var/folders/7w/p9vq5jld0ys_125td69sqz_h0000gn/T/ipykernel_4484/342759256.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['DEMAND_FORECAST'] = 0.0
/Users/keziya/Desktop/Github/Time-Series-Analysis---Electricity-Demand/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency informat

Training details:
Train set size 96170
Predicting demand for timestamp: 2023-06-27 14:30:00
Training details:
Train set size 96171
Predicting demand for timestamp: 2023-06-27 15:00:00


/Users/keziya/Desktop/Github/Time-Series-Analysis---Electricity-Demand/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 30min will be used.
  self._init_dates(dates, freq)
/Users/keziya/Desktop/Github/Time-Series-Analysis---Electricity-Demand/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 30min will be used.
  self._init_dates(dates, freq)


Training details:
Train set size 96172
Predicting demand for timestamp: 2023-06-27 15:30:00


/Users/keziya/Desktop/Github/Time-Series-Analysis---Electricity-Demand/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 30min will be used.
  self._init_dates(dates, freq)
/Users/keziya/Desktop/Github/Time-Series-Analysis---Electricity-Demand/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 30min will be used.
  self._init_dates(dates, freq)


...

/Users/keziya/Desktop/Github/Time-Series-Analysis---Electricity-Demand/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 30min will be used.
  self._init_dates(dates, freq)
/Users/keziya/Desktop/Github/Time-Series-Analysis---Electricity-Demand/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 30min will be used.
  self._init_dates(dates, freq)


Training details:
Train set size 96188
Predicting demand for timestamp: 2023-06-27 23:30:00


/Users/keziya/Desktop/Github/Time-Series-Analysis---Electricity-Demand/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 30min will be used.
  self._init_dates(dates, freq)
/Users/keziya/Desktop/Github/Time-Series-Analysis---Electricity-Demand/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 30min will be used.
  self._init_dates(dates, freq)


Training details:
Train set size 96189
Predicting demand for timestamp: 2023-06-28 00:00:00


/Users/keziya/Desktop/Github/Time-Series-Analysis---Electricity-Demand/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 30min will be used.
  self._init_dates(dates, freq)
/Users/keziya/Desktop/Github/Time-Series-Analysis---Electricity-Demand/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 30min will be used.
  self._init_dates(dates, freq)


In [122]:
test[['TOTALDEMAND','DEMAND_FORECAST']][:5]

,TOTALDEMAND,DEMAND_FORECAST
SETTLEMENTDATE,,
2023-06-27 14:30:00,8061.935000,7978.915930
2023-06-27 15:00:00,8387.033333,8134.542344
2023-06-27 15:30:00,8805.440000,8262.035565
2023-06-27 16:00:00,9383.706667,8366.481340
2023-06-27 16:30:00,10004.191667,8452.046045


##### Observations
1. Results are poor because the model predictions are added as ground truth to the original training set to get predictions for successive timestamps 
2. If the actual demand is passed as ground truth to predict the future demand, the result would be better

In [ ]:
# HOW TO CONSIDER OTHER FEATURES WHEN FORECASTING THE TIME SERIES VARIABLE